In [ ]:

import math
import random

# ========= 1) Box-Müller=========
def box_muller_pair():

    U1 = random.random()
    while U1 == 0.0:
        U1 = random.random()
    U2 = random.random()

    r = math.sqrt(-2.0 * math.log(U1))
    theta = 2.0 * math.pi * U2
    Z1 = r * math.cos(theta)
    Z2 = r * math.sin(theta)
    return Z1, Z2, U1, U2

SIGMA_X = 600.0  # horizontal
SIGMA_Y = 300.0  # vertical

def impacto():

    z1, z2, u1, u2 = box_muller_pair()   # N(0,1)
    x = 0.0 + SIGMA_X * z1       # x' = μx + σx * z1  (μx = 0)
    y = 0.0 + SIGMA_Y * z2       # y' = μy + σy * z2  (μy = 0)
    return x, y, z1, z2, u1, u2

def punto_en_poligono(x, y, poli):

    dentro = False
    n = len(poli)
    for i in range(n):
        x1, y1 = poli[i]
        x2, y2 = poli[(i + 1) % n]
        cond_y = ((y1 > y) != (y2 > y))
        if cond_y:
            x_int = x1 + (y - y1) * (x2 - x1) / (y2 - y1)
            if x_int >= x:  # rayo hacia +x
                dentro = not dentro
    return dentro


POLIGONO = [(-1000, -500), (1000, -500), (1000, 500), (-1000, 500)]

def simular_escuadron(n_bombas=10, semilla=None, imprimir=True):
    if semilla is not None:
        random.seed(semilla)
    exitos = 0
    resultados = []
    for i in range(1, n_bombas + 1):
        x, y, z1, z2, u1, u2 = impacto()
        hit = punto_en_poligono(x, y, POLIGONO)
        exitos += int(hit)
        resultados.append((i, x, y, z1, z2, u1, u2, "ÉXITO" if hit else "FALLO"))
    if imprimir:
        print("Simulación manual — 1 escuadrón")
        print(" N |        x (m)       |       y (m)       |      Z1      |      Z2      |      U1      |      U2      | Resultado")
        print("---+---------------------+-------------------+--------------+--------------+--------------+--------------+----------")
        for i, x, y, z1, z2, u1, u2, txt in resultados:
            print(f"{i:2d} | {x:>19.2f} | {y:>17.2f} | {z1:>12.4f} | {z2:>12.4f} | {u1:>12.4f} | {u2:>12.4f} | {txt}")
        print(f"\nAciertos: {exitos}/{n_bombas}  ->  {100*exitos/n_bombas:.1f}%\n")
    return exitos, resultados

def monte_carlo(n_escuadrones=10000, n_bombas=10, semilla=None):
    if semilla is not None:
        random.seed(semilla)
    total_bombas = n_escuadrones * n_bombas
    total_exitos = 0
    for _ in range(n_escuadrones):
        ex, _ = simular_escuadron(n_bombas=n_bombas, imprimir=False)
        total_exitos += ex

    p_hat = total_exitos / total_bombas

    z = 1.96
    se = math.sqrt(p_hat * (1 - p_hat) / total_bombas)
    li = max(0.0, p_hat - z * se)
    ls = min(1.0, p_hat + z * se)
    
    print(f"Simulación Monte Carlo:")
    print(f"Escuadrones: {n_escuadrones}, Bombas por escuadrón: {n_bombas}")
    print(f"Total de bombas: {total_bombas}")
    print(f"Probabilidad estimada: {p_hat:.4f}")
    print(f"Intervalo de confianza 95%: [{li:.4f}, {ls:.4f}]")
    
    return p_hat, li, ls




if __name__ == "__main__":
    simular_escuadron(n_bombas=10, semilla=42, imprimir=True)

    res = monte_carlo(n_escuadrones=20000, n_bombas=10, semilla=123)



Simulación manual — 1 escuadrón
 N |        x (m)       |       y (m)       |      Z1      |      Z2      | Resultado
---+---------------------+-------------------+--------------+--------------+----------
 1 |              560.43 |             44.40 |       0.9341 |       0.1480 | ÉXITO
 2 |              161.51 |            475.22 |       0.2692 |       1.5841 | ÉXITO
 3 |             -208.58 |           -210.20 |      -0.3476 |      -0.7007 | ÉXITO
 4 |              244.90 |             74.44 |       0.4082 |       0.2481 | ÉXITO
 5 |              774.46 |             73.36 |       1.2908 |       0.2445 | ÉXITO
 6 |            -1045.66 |            -17.60 |      -1.7428 |      -0.0587 | FALLO
 7 |              510.74 |            766.85 |       0.8512 |       2.5562 | FALLO
 8 |             -534.98 |            -77.61 |      -0.8916 |      -0.2587 | ÉXITO
 9 |             -883.56 |           -277.51 |      -1.4726 |      -0.9250 | ÉXITO
10 |              389.84 |              7.96 |  